In [10]:
import numpy as np
import crocoddyl
import pinocchio

from robot_properties_solo.config import Solo12Config


In [7]:
class DifferentialFwdKinematics(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self, state, actuation, costModel):
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, state.nv, costModel.nr)
        self.costs = costModel
        self.no_states = self.state.nq + self.state.nv

    def calc(self, data, x, u=None):
        if u is None:
            u = self.unone
        q, v = x[:self.state.nq], x[-self.state.nv:]
                    
        pinocchio.forwardKinematics(self.state.pinocchio, data.pinocchio, q, v)
        pinocchio.updateFramePlacements(self.state.pinocchio, data.pinocchio)
        pinocchio.centerOfMass(self.state.pinocchio, data.pinocchio, q, v)
        pinocchio.computeCentroidalMomentum(self.state.pinocchio, data.pinocchio)
        data.xout = u
        self.costs.calc(data.costs, x, u)
        data.cost = data.costs.cost

    def calcDiff(self, data, x, u=None):
        q, v = x[:self.state.nq], x[-self.state.nv:]
        if u is None:
            u = self.unone
        if True:
            self.calc(data, x, u)
        
        # u_a = np.concatenate((np.zeros(6), u))
        pinocchio.computeForwardKinematicsDerivatives(self.state.pinocchio, data.pinocchio, q, v, u)
        pinocchio.jacobianCenterOfMass(self.state.pinocchio, data.pinocchio)
        pinocchio.computeCentroidalDynamicsDerivatives(self.state.pinocchio, data.pinocchio, q, v, u)
        data.Fx = np.zeros((self.state.nv,self.no_states))
        data.Fu = np.eye(self.state.nv)
        # Computing the cost derivatives
        self.costs.calcDiff(data.costs, x, u)
        
    def createData(self):
        data = crocoddyl.DifferentialActionModelAbstract.createData(self)
        data.pinocchio = pinocchio.Data(self.state.pinocchio)
        data.costs = self.costs.createData(crocoddyl.DataCollectorMultibody(data.pinocchio))
        data.costs.shareMemory(data) # this allows us to share the memory of cost-terms of action model
        return data

In [8]:
class InverseKinematics():

    def __init__(self, rmodel, dt, T):
        """
        This class handles the inverse kinematics for the plan with crocoddyl
        Input:
            rmodel : pinocchio robot model
            dt : discrertization of time
            T : horizon of plan in seconds
        """
        self.rmodel = rmodel
        self.rdata = rmodel.createData()

        self.state = crocoddyl.StateMultibody(rmodel)
        self.actuation = crocoddyl.ActuationModelFloatingBase(self.state)
        self.dt = dt
        self.T = T
        self.N = int(T/dt)
        # This is the array to which all costs in each time step is added
        self.rcost_model_arr = []
        for n in range(self.N):
            # rCostModel = crocoddyl.CostModelSum(self.state, self.actuation.nu)
            rCostModel = crocoddyl.CostModelSum(self.state)

            self.rcost_model_arr.append(rCostModel)
        # This is the cost array that is passed into ddp solver
        self.rcost_arr = []

        # terminal cost model
        # self.terminalCostModel = crocoddyl.CostModelSum(self.state, self.actuation.nu)
        self.terminalCostModel = crocoddyl.CostModelSum(self.state)

    def setup_costs(self):
        """
        This function makes preapres the cost arrays so that they can be provided to
        crocoddyl to be solved
        Input:
            terminalCostModel : terminal cost model for DDP
        """
        for n in range(self.N):
            runningModel = crocoddyl.IntegratedActionModelEuler(
                DifferentialFwdKinematics(self.state, self.actuation, self.rcost_model_arr[n]), self.dt)
            self.rcost_arr.append(runningModel)

        self.terminalModel = crocoddyl.IntegratedActionModelEuler(
            DifferentialFwdKinematics(self.state, self.actuation, self.terminalCostModel), 0.)

    def optimize(self, x0):

        problem = crocoddyl.ShootingProblem(x0, self.rcost_arr, self.terminalModel)
        ddp = crocoddyl.SolverDDP(problem)
        log = crocoddyl.CallbackLogger()
        ddp.setCallbacks([log,
                        crocoddyl.CallbackVerbose(),
                        ])
        # Solving it with the DDP algorithm
        st = time.time()    
        ddp.solve()
        et = time.time()
        print("net time:", et - st)
        self.opt_sol = ddp.xs

        return self.opt_sol, ddp.us

In [ ]:
robot = Solo12Config.buildRobotWrapper()
robot_model = robot.model
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()